# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

# May need to install certain packages: pyproj, cartopy, geoviews onto kernel

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,mariupol,47.0667,37.5000,18.79,57,100,1.59,UA,1694462852
1,1,grytviken,-54.2811,-36.5092,-3.04,82,11,4.20,GS,1694462910
2,2,hermanus,-34.4187,19.2345,6.00,63,63,4.46,ZA,1694462664
3,3,margaret river,-33.9500,115.0667,17.06,78,68,7.43,AU,1694462910
4,4,utrik,11.2278,169.8474,28.97,79,42,9.01,MH,1694462910


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500
)
# Display the map
map_plot

# Map with very pretty colors for each city, a bit complicated to pull up

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_city_df = city_data_df[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21) \
    & (city_data_df["Wind Speed"] < 4.5) \
    & (city_data_df["Cloudiness"] == 0)
                               ]

# Drop any rows with null values
narrowed_city_df = narrowed_city_df.dropna()

# Display sample data
narrowed_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
22,22,guaraniacu,-25.1008,-52.8781,25.66,53,0,2.47,BR,1694462914
44,44,jamestown,42.0970,-79.2353,25.01,59,0,3.60,US,1694462815
79,79,siwa oasis,29.2041,25.5195,25.91,51,0,4.10,EG,1694462926
86,86,morondava,-20.2833,44.2833,23.82,86,0,0.93,MG,1694462927
128,128,prado,-17.3411,-39.2208,24.02,74,0,2.88,BR,1694462887
193,193,colonia,50.9333,6.9500,24.97,73,0,1.03,DE,1694462839
194,194,dinanagar,32.1500,75.4667,26.68,76,0,2.08,IN,1694462957
203,203,taloqan,36.7361,69.5345,21.35,43,0,1.64,AF,1694462959
204,204,dasht-e archi,37.1333,69.1667,23.73,37,0,2.40,AF,1694462959
260,260,limassol,34.6750,33.0333,26.04,78,0,2.06,CY,1694462974


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
22,guaraniacu,BR,-25.1008,-52.8781,53,
44,jamestown,US,42.0970,-79.2353,59,
79,siwa oasis,EG,29.2041,25.5195,51,
86,morondava,MG,-20.2833,44.2833,86,
128,prado,BR,-17.3411,-39.2208,74,
193,colonia,DE,50.9333,6.9500,73,
194,dinanagar,IN,32.1500,75.4667,76,
203,taloqan,AF,36.7361,69.5345,43,
204,dasht-e archi,AF,37.1333,69.1667,37,
260,limassol,CY,34.6750,33.0333,78,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": 20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

# 14 hotels were found!

Starting hotel search
guaraniacu - nearest hotel: No hotel found
jamestown - nearest hotel: DoubleTree Jamestown
siwa oasis - nearest hotel: فندق الكيلانى
morondava - nearest hotel: Hotel Sharon Menabe
prado - nearest hotel: Pousada Recanto do Prado
colonia - nearest hotel: Wasserturm Hotel Cologne
dinanagar - nearest hotel: No hotel found
taloqan - nearest hotel: No hotel found
dasht-e archi - nearest hotel: No hotel found
limassol - nearest hotel: Cornaro Fine Arts Center hotel
mao - nearest hotel: Eurohotel
bandrele - nearest hotel: Les Baobabs
ceccano - nearest hotel: Hotel La Villa
corinto - nearest hotel: Apollon
argostolion - nearest hotel: Ainos Hotel
shache - nearest hotel: 莎車新盛賓舘
aral - nearest hotel: No hotel found
sirte - nearest hotel: فندق المدينة-سرت
al hamul - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
22,guaraniacu,BR,-25.1008,-52.8781,53,No hotel found
44,jamestown,US,42.0970,-79.2353,59,DoubleTree Jamestown
79,siwa oasis,EG,29.2041,25.5195,51,فندق الكيلانى
86,morondava,MG,-20.2833,44.2833,86,Hotel Sharon Menabe
128,prado,BR,-17.3411,-39.2208,74,Pousada Recanto do Prado
193,colonia,DE,50.9333,6.9500,73,Wasserturm Hotel Cologne
194,dinanagar,IN,32.1500,75.4667,76,No hotel found
203,taloqan,AF,36.7361,69.5345,43,No hotel found
204,dasht-e archi,AF,37.1333,69.1667,37,No hotel found
260,limassol,CY,34.6750,33.0333,78,Cornaro Fine Arts Center hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    size="Humidity",
    scale=1,
    color="City",
    alpha=0.5,
    tiles="OSM",
    frame_width=700,
    frame_height=500,
    hover_cols=["Hotel Name", "Country"]
)

# Display the map
map_plot

# The map looks nicer with less cities.

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)